### **BPIC 2012 - No milestone**


In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from data.processor import *
from data.args import *

from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical

### Define the experiment parameters

In [2]:
milestone = 'All' #'A_PREACCEPTED' # 'W_Nabellen offertes', 'All'
subset = "W"
experiment =  'OHE'#'Standard'#'OHE', 'No_loops'
n_size = 5
max_size = 1000 #  3, 5, 10, 15, 20, 30, 50, 95
min_size = 0  # 0, 3, 5, 10, 15, 20, 30, 50

### Setting up the key parameters


In [3]:
# For local environment only
MY_WORKSPACE_DIR = os.path.join(os.getcwd(),'BPIC12')
MILESTONE_DIR = os.path.join(os.path.join(MY_WORKSPACE_DIR, milestone),experiment)

In [4]:
args = get_parameters('bpic12', MILESTONE_DIR, MY_WORKSPACE_DIR, milestone, experiment,n_size)
args['file_name_W_all'] = os.path.join(os.path.join(MY_WORKSPACE_DIR, "Translated_dataset"), "bpic12_translated_completed_W_all.csv" )

### Data Preprocessing

In [5]:
log_df = pd.read_csv(args['file_name_W_all'])
log_df = log_df.reset_index(drop=True)

log_df.head()

prefix_id  caseid                                          task  \
0   173736_6  173736  W_Filling in information for the application   
1   173736_4  173736  W_Filling in information for the application   
2   173736_5  173736  W_Filling in information for the application   
3  173736_12  173736  W_Filling in information for the application   
4   173736_1  173736  W_Filling in information for the application   

              role                end_timestamp                  trace_start  \
0  role_     11019  2011-10-01 23:40:09.6220000  2011-10-01 23:40:09.6220000   
1  role_     11019  2011-10-01 23:40:09.6220000  2011-10-01 23:40:09.6220000   
2  role_     11019  2011-10-01 23:40:09.6220000  2011-10-01 23:40:09.6220000   
3  role_     11019  2011-10-01 23:40:09.6220000  2011-10-01 23:40:09.6220000   
4  role_     11019  2011-10-01 23:40:09.6220000  2011-10-01 23:40:09.6220000   

   timelapsed                                      next_activity  \
0           0  W_Calling to add missing information to the ap...   
1           0                        W_Calling after sent offers   
2           0                        W_Assessing the application   
3           0                        W_Assessing the application   
4           0       W_Filling in information for the application   

                                           milestone  milestone_id  \
0                        W_Assessing the application             6   
1                        W_Calling after sent offers             4   
2                        W_Calling after sent offers             5   
3  W_Calling to add missing information to the ap...            12   
4       W_Filling in information for the application             1   

   task_index  prefix_length  
0           1              6  
1           1              4  
2           1              5  
3           1             12  
4           1              1

Milestone Extraction

We expect to filter the prefixes by the milestone, and train the model for each milestone separately to observe if there will be an improvement in the model explainability as well as performance


In [6]:
if milestone != 'All':
  log_df = log_df[log_df['milestone'] == milestone]
else:
  log_df = log_df[(log_df['prefix_length'] > min_size) & (log_df['prefix_length'] <= max_size) ]

In [7]:
# Replacing empty cells of categorical columns with 'none'
log_df = log_df.fillna('none')

Index creating for data encoding
Function : `create_index(log_df, column)`, Author: Manuel Camargo, Edited by: Bemali Wickramanayake

In [8]:
# Index creation for activity
ac_index = create_index(log_df, 'task')
index_ac = {v: k for k, v in ac_index.items()}

# Index creation for unit
rl_index = create_index(log_df, 'role')
index_rl = {v: k for k, v in rl_index.items()}

# Index creation for next activity
ne_index = create_index(log_df, 'next_activity')
index_ne = {v: k for k, v in ne_index.items()}

#mapping the dictionary values as columns in the dataframe
log_df['ac_index'] = log_df['task'].map(ac_index)
log_df['rl_index'] = log_df['role'].map(rl_index)
log_df['ne_index'] = log_df['next_activity'].map(ne_index)

log_df.head()

prefix_id  caseid                                          task  \
0   173736_6  173736  W_Filling in information for the application   
1   173736_4  173736  W_Filling in information for the application   
2   173736_5  173736  W_Filling in information for the application   
3  173736_12  173736  W_Filling in information for the application   
4   173736_1  173736  W_Filling in information for the application   

              role                end_timestamp                  trace_start  \
0  role_     11019  2011-10-01 23:40:09.6220000  2011-10-01 23:40:09.6220000   
1  role_     11019  2011-10-01 23:40:09.6220000  2011-10-01 23:40:09.6220000   
2  role_     11019  2011-10-01 23:40:09.6220000  2011-10-01 23:40:09.6220000   
3  role_     11019  2011-10-01 23:40:09.6220000  2011-10-01 23:40:09.6220000   
4  role_     11019  2011-10-01 23:40:09.6220000  2011-10-01 23:40:09.6220000   

   timelapsed                                      next_activity  \
0           0  W_Calling to add missing information to the ap...   
1           0                        W_Calling after sent offers   
2           0                        W_Assessing the application   
3           0                        W_Assessing the application   
4           0       W_Filling in information for the application   

                                           milestone  milestone_id  \
0                        W_Assessing the application             6   
1                        W_Calling after sent offers             4   
2                        W_Calling after sent offers             5   
3  W_Calling to add missing information to the ap...            12   
4       W_Filling in information for the application             1   

   task_index  prefix_length  ac_index  rl_index  ne_index  
0           1              6         5        35         3  
1           1              4         5        35         2  
2           1              5         5        35         1  
3           1             12         5        35         1  
4           1              1         5        35         4

### Experiments

#### Load the Required Libraries

In [9]:
import os
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import callbacks, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from pyflowchart import *
import plotly.express as px

from models.shared import *
from models.specialised import *
from models.explain import *

## Replicate Experiments

In [1]:
shared_acc = []
shared_prec = []
shared_rec = []
shared_f1 = []
runtime_shared = []
predict_shared = []

specialised_acc = []
specialised_prec = []
specialised_rec = []
specialised_f1 = []
runtime_specialised = []
predict_specialised = []

# Run the shared and specialised models 5 times each, to in the end compute the average performance scores
# for checking consistency
for i in list(range(5)):
    
    #### PRE-PROCESSING FOR EXPERIMENT
    
    # Randomize the data for checking consistency
    # To replicate without shuffling, comment the line below and change 'randomized_data' 'to log_df' in the succeeding line
    randomized_data = shuffle(log_df, random_state=i)

    # Split validation datasets
    log_df_train, log_df_test = split_train_test(randomized_data, 0.3) # 70%/30%
    
    # Normalize Numerical Features (timelapsed)
    numerical_features = ['timelapsed']
    log_df_train = normalize_events(log_df_train,args,numerical_features)
    log_df_test = normalize_events(log_df_test,args,numerical_features)
    
    # Reformat events: converting the dataframe into a dictionary to aid vectorization
    log_train = reformat_events(log_df_train, ac_index, rl_index,ne_index)
    log_test = reformat_events(log_df_test, ac_index, rl_index,ne_index)
    
    ### Vectorization: Convert the data dictionaries into trainable multi dimensional tensors
    
    # Obtain the maximum trc_len and cases for each set
    trc_len_train, cases_train = lengths(log_train)
    trc_len_test, cases_test = lengths(log_test)
    
    trc_len = max([trc_len_train, trc_len_test])
    
    
    # Converting the logs (dictionary) to a 3D-Tensor
    vec_train = vectorization(log_train,ac_index, rl_index, ne_index,trc_len,cases_train)
    vec_test = vectorization(log_test,ac_index, rl_index, ne_index,trc_len,cases_test)
    
    
    # Generating Initial Embedding Weights for the models
    ac_weights = to_categorical(sorted(index_ac.keys()), num_classes = len(ac_index))
    ac_weights[0] = 0 # embedding weights for label none = 0
    
    rl_weights =  to_categorical(sorted(index_rl.keys()), num_classes = len(rl_index))
    rl_weights[0] = 0 # embeddig weights for label none = 0
    
    
    # Converting the weights 
    weights = {'ac_weights':ac_weights, 'rl_weights':rl_weights, 'next_activity':len(ne_index)}
    
    # Converting the weights into a dictionary
    indexes = {'index_ac':index_ac, 'index_rl':index_rl,'index_ne':index_ne}
    
    # Converting the weights into a dictionary
    pre_index = {'ac_index':ac_index, 'rl_index':rl_index,'ne_index':ne_index}
    
    
    #### TRAINING AND TESTING SHARED MODEL
    
    print(f"Shared model {i+1}")
    
    start_shared = time.time()
    
    shared = shared_model(vec_train, weights, indexes, pre_index, args)
    shared.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    
    batch_size = 128 # 32, 64, 128, 256
    epochs = 250
    
    shared_history = shared_model_fit(vec_train, shared, indexes, pre_index, MY_WORKSPACE_DIR, batch_size, epochs, args)
    
    # Compute training time and append to list
    training_time_shared = time.time() - start_shared
    runtime_shared.append(training_time_shared)
    
    x_test,y_test = generate_inputs_shared(vec_test,args,indexes)
    
    accuracy = shared.evaluate(x_test, y_test, batch_size=batch_size)[1]
    
    # Append accuracy to list
    shared_acc.append(accuracy)
    
    # Compute time of making prediction given trained model
    start_shared_pred = time.time()
    y_pred_shared = shared.predict(x_test)
    predict_time_shared = (time.time() - start_shared_pred) / len(x_test)
    predict_shared.append(predict_time_shared)
    
    prec_rec_f1 = precision_recall_fscore_support(y_test.argmax(axis=1), y_pred_shared.argmax(axis=1), average='weighted')[0:3]
    precision, recall, f1_score = prec_rec_f1
    
    # Append precision, recall, and f1 scores to lists
    shared_prec.append(precision)
    shared_rec.append(recall)
    shared_f1.append(f1_score)
    
    # trained_model = os.path.join(os.path.join(MILESTONE_DIR, 'trained_models_gijs'),'shared_model_'+str(i)+subset+'.h5')
    # shared.save(trained_model)
    
    
    # Specialised model
    print("")
    print(f"Specialised model {i+1}")
    
    start_specialised = time.time()
    
    specialised = specialised_model(vec_train, weights, indexes, pre_index, args)
    specialised.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    
    batch_size = 256 # 32, 64, 128, 256
    epochs = 200
    
    specialised_history = specialised_model_fit(vec_train, specialised, indexes, pre_index, MY_WORKSPACE_DIR, batch_size, epochs, args)
    
    # Compute training time and append to list
    training_time_specialised = time.time() - start_specialised
    runtime_specialised.append(training_time_specialised)
    
    x_test,y_test = generate_inputs(vec_test,args,indexes, experiment)
    
    accuracy = specialised.evaluate(x_test, y_test, batch_size=batch_size)[1]
    
    # Append accuracy to list
    specialised_acc.append(accuracy)
    
    # Compute time of making prediction given trained model
    start_specialised_pred = time.time()
    y_pred_specialised = specialised.predict(x_test)
    predict_time_specialised = (time.time() - start_specialised_pred) / len(x_test)
    predict_specialised.append(predict_time_specialised)
    
    prec_rec_f1 = precision_recall_fscore_support(y_test.argmax(axis=1), y_pred_specialised.argmax(axis=1), average='weighted')[0:3]
    precision, recall, f1_score = prec_rec_f1
    
    # Append precision, recall, and f1 scores to lists
    specialised_prec.append(precision)
    specialised_rec.append(recall)
    specialised_f1.append(f1_score)
    
    # trained_model = os.path.join(os.path.join(MILESTONE_DIR, 'trained_models_gijs'),'specialised_model_'+str(i)+subset+'.h5')
    # specialised.save(trained_model)

### Get table of performance metrics

#### No Shuffling

In [21]:
shared_accuracy = np.mean(shared_acc)
shared_precision = np.mean(shared_prec)
shared_recall = np.mean(shared_rec)
shared_f1_score = np.mean(shared_f1)
runtime_shared_train = np.mean(runtime_shared)
runtime_predict_shared = np.mean(predict_shared)

specialised_accuracy = np.mean(specialised_acc)
specialised_precision = np.mean(specialised_prec)
specialised_recall = np.mean(specialised_rec)
specialised_f1_score = np.mean(specialised_f1)
runtime_specialised_train = np.mean(runtime_specialised)
runtime_predict_specialised = np.mean(predict_specialised)

accuracy = [shared_accuracy, specialised_accuracy]
precision = [shared_precision, specialised_precision]
recall = [shared_recall, specialised_recall]
f1 = [shared_f1_score, specialised_f1_score]
train_time = [runtime_shared_train, runtime_specialised_train]
pred_time = [runtime_predict_shared, runtime_predict_specialised]

In [22]:
std_shared_accuracy = round(np.std(shared_acc), 4)
std_shared_precision = round(np.std(shared_prec), 4)
std_shared_recall = round(np.std(shared_rec), 4)
std_shared_f1_score = round(np.std(shared_f1), 4)
std_runtime_shared_train = round(np.std(runtime_shared), 4)
std_runtime_predict_shared = round(np.std(predict_shared), 4)


std_specialised_accuracy = round(np.std(specialised_acc), 4)
std_specialised_precision = round(np.std(specialised_prec), 4)
std_specialised_recall = round(np.std(specialised_rec), 4)
std_specialised_f1_score = round(np.std(specialised_f1), 4)
std_runtime_specialised_train = round(np.std(runtime_specialised), 4)
std_runtime_predict_specialised = round(np.std(predict_specialised), 4)

std_accuracy = [std_shared_accuracy, std_specialised_accuracy]
std_precision = [std_shared_precision, std_specialised_precision]
std_recall = [std_shared_recall, std_specialised_recall]
std_f1 = [std_shared_f1_score, std_specialised_f1_score]
std_train_time = [std_runtime_shared_train, std_runtime_specialised_train]
std_pred_time = [std_runtime_predict_shared, std_runtime_predict_specialised]

In [24]:
performance_metrics = pd.DataFrame({"Model":["Shared", "Specialised"], "Event log":["BPIC 2012 W", "BPIC 2012 W"], "Accuracy": accuracy, "std Accuracy":std_accuracy, "Precision": precision, "std Precision": std_precision, "Recall": recall, "std Recall": std_recall, "F1-Score": f1, "std F1-Score": std_f1, "Training Time": train_time, "std Training Time": std_train_time, "Prediction Time": pred_time, "std Prediction Time": std_pred_time})
performance_metrics

Model    Event log  Accuracy  std Accuracy  Precision  std Precision  \
0       Shared  BPIC 2012 W  0.846870        0.0010   0.854653         0.0015   
1  Specialised  BPIC 2012 W  0.837347        0.0022   0.843491         0.0020   

     Recall  std Recall  F1-Score  std F1-Score  Training Time  \
0  0.846870      0.0010  0.841643        0.0016     353.156080   
1  0.837347      0.0022  0.829092        0.0027    1068.389835   

   std Training Time  Prediction Time  std Prediction Time  
0           124.6339        15.116274              17.5978  
1           233.1504        25.647541              28.9599

#### Shuffling

In [17]:
shared_accuracy = np.mean(shared_acc)
shared_precision = np.mean(shared_prec)
shared_recall = np.mean(shared_rec)
shared_f1_score = np.mean(shared_f1)
runtime_shared_train = np.mean(runtime_shared)
runtime_predict_shared = np.mean(predict_shared)

specialised_accuracy = np.mean(specialised_acc)
specialised_precision = np.mean(specialised_prec)
specialised_recall = np.mean(specialised_rec)
specialised_f1_score = np.mean(specialised_f1)
runtime_specialised_train = np.mean(runtime_specialised)
runtime_predict_specialised = np.mean(predict_specialised)

accuracy = [shared_accuracy, specialised_accuracy]
precision = [shared_precision, specialised_precision]
recall = [shared_recall, specialised_recall]
f1 = [shared_f1_score, specialised_f1_score]
train_time = [runtime_shared_train, runtime_specialised_train]
pred_time = [runtime_predict_shared, runtime_predict_specialised]

In [18]:
std_shared_accuracy = round(np.std(shared_acc), 4)
std_shared_precision = round(np.std(shared_prec), 4)
std_shared_recall = round(np.std(shared_rec), 4)
std_shared_f1_score = round(np.std(shared_f1), 4)
std_runtime_shared_train = round(np.std(runtime_shared), 4)
std_runtime_predict_shared = round(np.std(predict_shared), 4)


std_specialised_accuracy = round(np.std(specialised_acc), 4)
std_specialised_precision = round(np.std(specialised_prec), 4)
std_specialised_recall = round(np.std(specialised_rec), 4)
std_specialised_f1_score = round(np.std(specialised_f1), 4)
std_runtime_specialised_train = round(np.std(runtime_specialised), 4)
std_runtime_predict_specialised = round(np.std(predict_specialised), 4)

std_accuracy = [std_shared_accuracy, std_specialised_accuracy]
std_precision = [std_shared_precision, std_specialised_precision]
std_recall = [std_shared_recall, std_specialised_recall]
std_f1 = [std_shared_f1_score, std_specialised_f1_score]
std_train_time = [std_runtime_shared_train, std_runtime_specialised_train]
std_pred_time = [std_runtime_predict_shared, std_runtime_predict_specialised]

In [19]:
performance_metrics = pd.DataFrame({"Model":["Shared", "Specialised"], "Event log":["BPIC 2012 W", "BPIC 2012 W"], "Accuracy": accuracy, "std Accuracy":std_accuracy, "Precision": precision, "std Precision": std_precision, "Recall": recall, "std Recall": std_recall, "F1-Score": f1, "std F1-Score": std_f1, "Training Time": train_time, "std Training Time": std_train_time, "Prediction Time": pred_time, "std Prediction Time": std_pred_time})
performance_metrics

Model    Event log  Accuracy  std Accuracy  Precision  std Precision  \
0       Shared  BPIC 2012 A  0.878299        0.0032   0.881953         0.0036   
1  Specialised  BPIC 2012 A  0.857154        0.0059   0.854418         0.0068   

     Recall  std Recall  F1-Score  std F1-Score  Training Time  \
0  0.878299      0.0032  0.878133        0.0028     405.861586   
1  0.857154      0.0059  0.852622        0.0061     942.308979   

   std Training Time  Prediction Time  std Prediction Time  
0           219.5016         5.412130               3.6695  
1           283.2446         5.145069               1.8584

##### Load Saved Model and Evaluate

In [ ]:
# shared_trained_model = os.path.join(os.path.join(MILESTONE_DIR, 'trained_models_gijs'),'shared_model_'+subset+'.h5')
# shared = load_model(shared_trained_model)

##### Load Saved Model and Evaluate

In [ ]:
# trained_model = os.path.join(os.path.join(MILESTONE_DIR, 'trained_models_gijs'),'specialised_model_'+subset+'.h5')
# specialised = load_model(trained_model)